In [23]:
import pandas as pd

In [30]:
train_data = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test_data = pd.read_csv('testData.tsv', header = 0, delimiter= "\t", quoting=3) 

In [25]:
from bs4 import BeautifulSoup
def clean_html_char(x):
    bs = BeautifulSoup(x)
    return bs.get_text()

In [26]:
import re
re_obj = re.compile('[^a-zA-Z]')
def keep_ascii_char(x):
    y = re_obj.sub(' ', x)
    return y

In [27]:
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words('english'))
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')
def nltk_clean_words(x):
    ss = x.split(' ')
    return filter(lambda x: x not in eng_stopwords and x, map(lambda x: stemmer.stem(x), ss))

In [28]:
def sentence_to_words(x):
    x = clean_html_char(x)
    x = keep_ascii_char(x)
    return nltk_clean_words(x)
def clean_review(x):
    words = sentence_to_words(x)
    return ' '.join(words)

In [31]:
train_reviews = map(lambda x: clean_review(x), train_data['review'])
test_reviews = map(lambda x: clean_review(x), test_data['review'])

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
# vectorizer = CountVectorizer(analyzer = "word", max_features = 5000)
vectorizer = TfidfVectorizer(analyzer="word", max_features=5000)
vectorizer.fit(train_reviews + test_reviews)
train_mx = vectorizer.transform(train_reviews)
test_mx = vectorizer.transform(test_reviews)

In [48]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators= 500, n_jobs = 2, oob_score= True)
rf.fit(train_mx, train['sentiment'])
print rf.oob_score_

0.84736


In [49]:
result = rf.predict(test_mx)

In [50]:
output = pd.DataFrame(data = {"id": test_data["id"], "sentiment": result})
output.to_csv('bag-of-words.csv', index = False, quoting=3)